In [33]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
import re
from datetime import datetime
from dotenv import load_dotenv
import os 

In [ ]:
load_dotenv()
server_ip = os.getenv("SERVER_IP")
server_port = os.getenv("SERVER_PORT")

# Initiate the connection to the database

In [18]:
# Connect to MongoDB
server_adress = f'{server_ip}:{server_port}' 
client = MongoClient(server_adress)

# Select database
db = client['dblp']

# Select collection
collection = db['awards']

# Queries

In [19]:
queries = []

queries.append([
    {
        '$group': {
            '_id': {
                '$toLower': '$institution.city_name'
            }, 
            'montant_cumul': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'montant_cumul': -1
        }
    }, {
        '$project': {
            'City Name': '$_id', 
            'Cumulative Amount': '$montant_cumul', 
            '_id': 0
        }
    }
])

In [20]:
queries.append([
    {
        '$match': {
            'institution.city_name': re.compile(r"New York(?i)")
        }
    }, {
        '$group': {
            '_id': {
                'instName': '$institution.name', 
                'instZip': '$institution.zipcode'
            }, 
            'quantity': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'quantity': -1
        }
    }, {
        '$project': {
            'Insitution name': '$_id.instName', 
            'Insitution zip': '$_id.instZip', 
            'Awards received': '$quantity', 
            '_id': 0
        }
    }
])

In [21]:
queries.append([
    {
        '$match': {
            'investigators.email_id': {
                '$ne': None
            }
        }
    }, {
        '$group': {
            '_id': '$investigators.email_id', 
            'setDomaines': {
                '$addToSet': '$foa_info'
            }, 
            'email': {
                '$first': '$investigators.email_id'
            }
        }
    }, {
        '$project': {
            'email': '$_id', 
            'Number of domains': {
                '$size': '$setDomaines'
            }, 
            '_id': 0
        }
    }, {
        '$sort': {
            'Number of domains': -1
        }
    }, {
        '$limit': 10
    }
])

In [22]:
queries.append([
    {
        '$match': {
            'institution.state_name': 'California'
        }
    }, {
        '$group': {
            '_id': '$organisation_code', 
            'nbInstitutions': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'nbInstitutions': -1
        }
    }, {
        '$limit': 5
    }
])

In [23]:
queries.append([
    {
        '$group': {
            '_id': '$institution', 
            'count': {
                '$sum': 1
            }, 
            'averageAwardAmount': {
                '$avg': '$amount'
            }, 
            'maxAwardAmount': {
                '$max': '$amount'
            }, 
            'minAwardAmount': {
                '$min': '$amount'
            }, 
            'stdAwardAmount': {
                '$stdDevPop': '$amount'
            }
        }
    }, {
        '$sort': {
            'count': -1
        }
    }
])

In [24]:
queries.append([
    {
        '$unwind': {
            'path': '$programs', 
            'includeArrayIndex': 'string', 
            'preserveNullAndEmptyArrays': False
        }
    }, {
        '$group': {
            '_id': '$programs', 
            'nb_amount': {
                '$sum': '$amount'
            }
        }
    }, {
        '$sort': {
            'nb_amount': -1
        }
    }, {
        '$project': {
            '_id': 1, 
            'nb_amount': 1
        }
    }
])

In [25]:
queries.append([
    {
        '$match': {
            'investigators.email_id': re.compile(r"(a)(?i)")
        }
    }, {
        '$group': {
            '_id': {
                'date': {
                    '$substr': [
                        '$effective_date', 6, -1
                    ]
                }, 
                'invest': '$investigators.email_id'
            }, 
            'nb': {
                '$sum': 1
            }
        }
    }, {
        '$project': {
            'Investigator': '$_id.invest', 
            'Date': '$_id.date', 
            'Number': '$nb', 
            '_id': 0
        }
    }
])

In [26]:
queries.append([
    {
        '$match': {
            'investigators.email_id': 'jeremygtaylor@compuserve.com'
        }
    }, {
        '$project': {
            'Email': '$investigators.email_id', 
            'date': {
                '$toDate': '$investigators.start_date'
            }, 
            'today': datetime.utcnow()
        }
    }, {
        '$addFields': {
            '_id': '$_id', 
            'dateDiff': {
                '$subtract': [
                    '$today', '$date'
                ]
            }
        }
    }, {
        '$project': {
            'Email': 1, 
            'dateDiff': 1, 
            'secs': {
                '$divide': [
                    '$dateDiff', 1000
                ]
            }
        }
    }, {
        '$project': {
            'Email': 1, 
            'mins': {
                '$divide': [
                    '$secs', 60
                ]
            }
        }
    }, {
        '$project': {
            'Email': 1, 
            'hours': {
                '$divide': [
                    '$mins', 60
                ]
            }
        }
    }, {
        '$project': {
            'Email': 1, 
            'days': {
                '$divide': [
                    '$hours', 24
                ]
            }
        }
    }, {
        '$group': {
            '_id': '$Email', 
            'avgDays': {
                '$avg': '$days'
            }
        }
    }, {
        '$project': {
            'Email': '$_id', 
            'Average days': '$avgDays', 
            '_id': 0
        }
    }
])

# Definitions

In [27]:
def get_aggregation_execTimeMs(query, db, collection):
    cursor = db.command(
        'explain', 
        {
            'aggregate': "awards", 
            'pipeline': query, 
            'cursor': {}
        }, 
        verbosity='executionStats'
    )
    times = []
    # Keep only the longest execution time
    for shard in cursor["shards"].keys():
        times.append(cursor["shards"][shard]["stages"][0]["$cursor"]["executionStats"]["executionTimeMillis"])
    return np.max(times)

In [28]:
def run_query_n_times(query, db, collection, n):
    times = []
    for i in range(n):
        t = get_aggregation_execTimeMs(query, db, collection)
        print(f"Ran in {t}ms")
        times.append(t)
    return times

In [29]:
def initialize_df(queries, n):
    times_list = [f"{i}_stats" for i in range(1, n+1)] # List of stats columns
    stats_list = ['min_runtime', 'max_runtime', 'all_avg_runtime']
    columns = ['query_number', 'query'] + times_list + stats_list
    
    df = pd.DataFrame(np.zeros((len(queries),len(columns))), columns=columns) # Create dataframe
    
    df['query_number'] = [i for i in range(1, len(queries)+1)]
    df['query'] = queries

    return df, times_list, stats_list

# Initialize a dataframe

In [30]:
n = 10 # Number of times to run each query
df, times_list, stats_list = initialize_df(queries, n)
df

,query_number,query,1_stats,2_stats,3_stats,4_stats,5_stats,6_stats,7_stats,8_stats,9_stats,10_stats,min_runtime,max_runtime,all_avg_runtime
0,1,[{'$group': {'_id': {'$toLower': '$institution...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,[{'$match': {'institution.city_name': re.compi...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,[{'$match': {'investigators.email_id': {'$ne':...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,[{'$match': {'institution.state_name': 'Califo...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,"[{'$group': {'_id': '$institution', 'count': {...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,"[{'$unwind': {'path': '$programs', 'includeArr...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,[{'$match': {'investigators.email_id': re.comp...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,[{'$match': {'investigators.email_id': 'jeremy...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Loop through the queries

In [31]:
num_shards = 4
for query in queries:
    query_number = queries.index(query) + 1
    print(f"Running query {query_number}")
    times = run_query_n_times(query, db, "awards", n)
    
    print("Times for query", query_number, ":", times)
    df.loc[df['query_number'] == query_number, times_list] = times
    df.loc[df['query_number'] == query_number, 'min_runtime'] = min(times)
    df.loc[df['query_number'] == query_number, 'max_runtime'] = max(times)
    df.loc[df['query_number'] == query_number, 'all_avg_runtime'] = np.mean(times)
    
    # Drop min and max values
    times.remove(min(times))
    times.remove(max(times))
    print("Times without outliers for query", query_number, ":", times)
    
    df.loc[df['query_number'] == query_number, 'avg_runtime_no_outliers'] = np.mean(times)
    
df.to_csv(f"results/S{num_shards}_results.csv", index=False)

Running query 1
Ran in 2451ms
Ran in 2446ms
Ran in 2450ms
Ran in 2459ms
Ran in 2464ms
Ran in 2473ms
Ran in 2471ms
Ran in 2452ms
Ran in 2446ms
Ran in 2461ms
Times for query 1 : [2451, 2446, 2450, 2459, 2464, 2473, 2471, 2452, 2446, 2461]
Times without outliers for query 1 : [2451, 2450, 2459, 2464, 2471, 2452, 2446, 2461]
Running query 2
Ran in 880ms
Ran in 881ms
Ran in 882ms
Ran in 879ms
Ran in 881ms
Ran in 882ms
Ran in 880ms
Ran in 879ms
Ran in 879ms
Ran in 881ms
Times for query 2 : [880, 881, 882, 879, 881, 882, 880, 879, 879, 881]
Times without outliers for query 2 : [880, 881, 881, 882, 880, 879, 879, 881]
Running query 3
Ran in 1143ms
Ran in 1135ms
Ran in 1143ms
Ran in 1278ms
Ran in 1140ms
Ran in 1139ms
Ran in 1145ms
Ran in 1168ms
Ran in 1138ms
Ran in 1135ms
Times for query 3 : [1143, 1135, 1143, 1278, 1140, 1139, 1145, 1168, 1138, 1135]
Times without outliers for query 3 : [1143, 1143, 1140, 1139, 1145, 1168, 1138, 1135]
Running query 4
Ran in 982ms
Ran in 980ms
Ran in 978ms
Ran 

In [32]:
df

,query_number,query,1_stats,2_stats,3_stats,4_stats,5_stats,6_stats,7_stats,8_stats,9_stats,10_stats,min_runtime,max_runtime,all_avg_runtime,avg_runtime_no_outliers
0,1,[{'$group': {'_id': {'$toLower': '$institution...,2451.0,2446.0,2450.0,2459.0,2464.0,2473.0,2471.0,2452.0,2446.0,2461.0,2446.0,2473.0,2457.3,2456.750
1,2,[{'$match': {'institution.city_name': re.compi...,880.0,881.0,882.0,879.0,881.0,882.0,880.0,879.0,879.0,881.0,879.0,882.0,880.4,880.375
2,3,[{'$match': {'investigators.email_id': {'$ne':...,1143.0,1135.0,1143.0,1278.0,1140.0,1139.0,1145.0,1168.0,1138.0,1135.0,1135.0,1278.0,1156.4,1143.875
3,4,[{'$match': {'institution.state_name': 'Califo...,982.0,980.0,978.0,979.0,977.0,976.0,977.0,978.0,978.0,977.0,976.0,982.0,978.2,978.000
4,5,"[{'$group': {'_id': '$institution', 'count': {...",3716.0,3695.0,3683.0,3766.0,3715.0,3730.0,3717.0,3729.0,3752.0,3753.0,3683.0,3766.0,3725.6,3725.875
5,6,"[{'$unwind': {'path': '$programs', 'includeArr...",2175.0,2161.0,2149.0,2169.0,2174.0,2166.0,2192.0,2184.0,2166.0,2168.0,2149.0,2192.0,2170.4,2170.375
6,7,[{'$match': {'investigators.email_id': re.comp...,1192.0,1188.0,1190.0,1194.0,1206.0,1208.0,1189.0,1188.0,1195.0,1205.0,1188.0,1208.0,1195.5,1194.875
7,8,[{'$match': {'investigators.email_id': 'jeremy...,4.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0,3.0,3.0,3.0,4.0,3.5,3.500
